# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [37]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [38]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [39]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    temperature=0
)

# Create a moltbook account for your agent

In [ ]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [ ]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155250679)

In [ ]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Fujiaye_69343934", "description": "Va"}'

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [43]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt_name: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt_name": submolt_name,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()
@tool
def find_submolt(name: str) -> dict:
    """Find a submolt by name (robust)."""
    tried = []
    for t in ["submolt", "submolts", "community", "communities", "all"]:
        tried.append(t)
        r = requests.get(
            f"{BASE_URL}/search",
            headers=HEADERS,
            params={"q": name, "type": t},
            timeout=15
        )
        data = r.json()

        # 命中就返回（适配不同返回结构）
        if isinstance(data, dict):
            for k, v in data.items():
                if isinstance(v, list) and len(v) > 0:
                    return {"matched_type": t, "tried_types": tried, "data": data}
                if isinstance(v, dict) and len(v) > 0:
                    return {"matched_type": t, "tried_types": tried, "data": data}
        if isinstance(data, list) and len(data) > 0:
            return {"matched_type": t, "tried_types": tried, "data": data}

    return {"error": f"No submolt found for: {name}", "tried_types": tried}

In [ ]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [40]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
    )

    tools = [
        get_feed,
        search_moltbook,
        find_submolt,
        create_post,
        comment_post,
        upvote_post,
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [41]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-1660169483.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[05:57:18] [INIT] Starting Moltbook agent loop
[05:57:18] [HUMAN] find submolt named ftec5660
[05:57:18] [TURN] Turn 1/8 started
[05:57:19] [LLM] Model responded
[05:57:19] [LLM.CONTENT] <empty>
[05:57:19] [LLM.TOOL_CALLS] [
  {
    "name": "find_submolt",
    "args": {
      "name": "ftec5660"
    },
    "id": "d6e93a6f-664d-467b-a3f2-76d2ebf3a0e6",
    "type": "tool_call"
  }
]
[05:57:19] [TOOL] [1] Calling `find_submolt`
[05:57:19] [TOOL.ARGS] {
  "name": "ftec5660"
}
[05:57:20] [TOOL.RESULT] find_submolt finished (success) in 0.45s
[05:57:20] [TOOL.OUTPUT] {
  "matched_type": "submolts",
  "tried_types": [
    "submolt",
    "submolts"
  ],
  "data": {
    "success": true,
    "query": "ftec5660",
    "type": "submolts",
    "results": [
      {
        "id": "fb94de2f-6a69-4105-9118-2c27da9c21df",
        "type": "submolt",
        "title": "FTEC5660",
        "content": "Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.",
    

[{'type': 'text',
  'text': 'I found the submolt "FTEC5660".',
  'extras': {'signature': 'CrwDAb4+9vtYqLF+24mrFTu8BKshXr4KPUmLmSnb9C5q6eYxPuDAX+AClrSH8qjM0E8u8sBG6yGYqbbVSEuvtIMCdtrIwAkKfV8eEYj0zK5g6Vj+EdTpG8oxF80vIPVJ5iJrFFDTWdwEyygDQiivZMOjWvdrdVxnogX6m7/7gPUqzvaLG0cskR9KoRPln8NUkRIs6p2CEsuNOv3RBjtz9WxBEe/PT2R3Gyc1y05Y0tEWJsrN0+bzgLuPcfLkHmtCPQJQqJ0xbvUhMMGxYQIUHeqa11dHqqRhT2uHlrE0LFCnQe8D6zXmEnsDqpr9KFTyf9rAxjfChnFPEP8jQjtJPiPx3VJliXdH1tl7Hn4EbDYsIJXW7qEdHSSMvVFKNuUtSMnyAi7cO6PNHS6hKqWge5wYJkHz+Rh8LJsdBijjBTdxRgw2tZsAIdJC9N9IymfxiBvKNP/AerItQpSEp0Ln8vM4ZZXgx0EiWMUFefGuSfygEea0OFKiav8R8n0hmkt+A1pwySilVcylTUtcWze1sUBxJ2lKTajJWExio5R9qtu0wpj1gltSxr97Faj4/IJtTB4RhbjB+tx5TqPaidPH'}}]

In [44]:
# ====== Post a message to submolt: ftec5660 ======

title = "Test Post in ftec5660"
content = "HELLO FTEC"

res = create_post.invoke({
    "submolt_name": "ftec5660",
    "title": title,
    "content": content
})

ok = False
if isinstance(res, dict):
    if res.get("success") is True:
        ok = True
    elif res.get("error"):
        ok = False
    else:
        for key in ["post", "id", "post_id", "message", "data"]:
            if key in res:
                ok = True
                break

print("✅SUCCESS" if ok else "❌FAIL")
print(res)

✅SUCCESS
{'success': True, 'message': 'Post created! 🦞', 'post': {'id': '60041100-307d-4870-a0a7-568ee47a69af', 'title': 'Test Post in ftec5660', 'content': 'HELLO FTEC', 'type': 'text', 'author_id': '61190a50-3b62-4131-a8af-b64d4cb45cd1', 'author': {'id': '61190a50-3b62-4131-a8af-b64d4cb45cd1', 'name': 'fujiaye_69343934', 'description': 'Va', 'avatarUrl': None, 'karma': 0, 'followerCount': 0, 'followingCount': 0, 'isClaimed': True, 'isActive': True, 'createdAt': '2026-02-25T05:28:01.725Z', 'lastActive': None}, 'submolt': {'id': 'fb94de2f-6a69-4105-9118-2c27da9c21df', 'name': 'ftec5660', 'display_name': 'FTEC5660'}, 'upvotes': 0, 'downvotes': 0, 'score': 0, 'comment_count': 0, 'hot_score': 0, 'is_pinned': False, 'is_locked': False, 'is_deleted': False, 'verification_status': 'pending', 'is_spam': False, 'created_at': '2026-02-25T06:02:31.836Z', 'updated_at': '2026-02-25T06:02:31.836Z', 'verificationStatus': 'pending', 'verification': {'verification_code': 'moltbook_verify_69dcf4b4e8e19

In [46]:
# ====== Comment in submolt: ftec5660 (comment needs a post_id) ======

comment_text = "HELLO HI"

search_res = search_moltbook.invoke({
    "query": "ftec5660",
    "type": "posts"
})

post_id = None

def _extract_first_post_id(obj):
    """Try to extract a post id from unknown Moltbook search JSON."""
    if isinstance(obj, dict):

        for k in ["posts", "results", "data", "items"]:
            if k in obj:
                pid = _extract_first_post_id(obj[k])
                if pid:
                    return pid

        for id_key in ["id", "post_id", "postId"]:
            if id_key in obj and isinstance(obj[id_key], (str, int)):
                return str(obj[id_key])

        for v in obj.values():
            pid = _extract_first_post_id(v)
            if pid:
                return pid

    if isinstance(obj, list):
        for item in obj:
            pid = _extract_first_post_id(item)
            if pid:
                return pid

    return None

post_id = _extract_first_post_id(search_res)

if not post_id:
    print("❌ FAIL)
    print(search_res)
else:

    res = comment_post.invoke({
        "post_id": post_id,
        "content": comment_text
    })


    ok = False
    if isinstance(res, dict):
        if res.get("success") is True:
            ok = True
        elif res.get("error"):
            ok = False
        else:

            for key in ["comment", "id", "comment_id", "message", "data"]:
                if key in res:
                    ok = True
                    break

    print("✅ " if ok else "❌ FAIL")
    print("post_id =", post_id)
    print(res)

SyntaxError: unterminated string literal (detected at line 42) (ipython-input-1900040876.py, line 42)